# Proyecto Matematicas Discretas

#### Profesor Leonardo Ignacio Martinez Sandoval
#### Ayudante Victor Hugo Almendra Hernandez

Alfonso Barajas Cervantes

In [1]:
import numpy as np
import pandas as pd
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple
import collections

# Optimizacion Costo-Nutricion Dieta Vegetariana Cruda

A continuación escribe el presupuesto semanal para comprar alimentos de la dieta Vegetariana Cruda (Sin símbolo de pesos, sólo número) =:

In [2]:
presupuesto_total = float(input())
#pruebe con 800,1000, 1200, 1600 (presupuesto menor a 8000 pesos) (Mas informacion cheque Suposiciones)

1000


Se procede a dividir el presupuesto para obtener el dinero que se destinará a las frutas, verduras y semillas oleaginosas. Con un $40 \%$ destinado a las frutas, un $40 \%$ a las semillas oleaginosas mientras que el restante $20 \%$ para las verduras.

In [3]:
def division_presupuesto(p):
    f = 0.4*p
    v = 0.2*p
    o = 0.4*p
    return f,v,o
W_f, W_v, W_o = division_presupuesto(presupuesto_total)

# Frutas

In [4]:
dff = pd.read_csv('fruits.csv', sep=',' ,encoding='latin-1') 
dff.head()

,id_fruits,fruits,price,value_f
0,1,AGUACATE,30,5
1,2,AGUACATE CRIOLLO,25,5
2,3,CAPULÍN,40,5
3,4,CEREZAS,50,5
4,5,CHABACANO,80,4


In [5]:
name_f = dff.iloc[0:53,1]
weight_f = dff.iloc[0:53,2]
value_f = dff.iloc[0:53,3]
name_fr = []
weight_fr = []
value_fr = []
for i in range(52):
    name_fr.append(name_f[i])
    value_fr.append(value_f[i])
    weight_fr.append(weight_f[i])
#name_fr
#weight_fr
#value_fr

# Verduras

In [6]:
dfv = pd.read_csv('veg_all.csv', sep=',' ,encoding='latin-1') 
dfv.head()

,id_veg,veg,price,value_f
0,54,ALCACHOFA,80,4
1,55,ACELGA,25,5
2,56,APIO,25,3
3,57,BERROS,20,4
4,58,BROCOLI,40,3


In [7]:
name_v = dfv.iloc[0:53,1]
weight_v = dfv.iloc[0:53,2]
value_v = dfv.iloc[0:53,3]
name_ve = []
weight_ve = []
value_ve = []
for i in range(53):
    name_ve.append(name_v[i])
    value_ve.append(value_v[i])
    weight_ve.append(weight_v[i])
#name_ve
#weight_ve
#value_ve

# Semillas Oleaginosas

In [8]:
dfo = pd.read_csv('oilseeds.csv', sep=',' ,encoding='latin-1') 
dfo.tail()

,id_oilseeds,oilseeds,price,value_f
9,113,NUEZ,250,5
10,114,NUEZ DE LA INDIA,400,4
11,115,PIÑÓN,500,4
12,116,PISTACHES,400,4
13,117,CALABAZA DE CASTILLA,130,4


In [9]:
name_o = dfo.iloc[0:14,1]
weight_o = dfo.iloc[0:14,2]
value_o = dfo.iloc[0:14,3]
name_oi = []
weight_oi = []
value_oi = []
for i in range(14):
    name_oi.append(name_o[i])
    value_oi.append(value_o[i])
    weight_oi.append(weight_o[i])
#name_oi
#weight_oi
#value_oi

## Knapsack 0/1 usando Programacion Dinamica

In [10]:
def knapsack_dp(W, wt, val, n): 
    # Matriz DP
    K = [[0 for w in range(W + 1)] 
            for i in range(n + 1)]
    # Lista para guardar indices de elemetos
    items = []
    
    # Se construye  la matriz K[][] desde abajo
    # hasta arriba
    for i in range(n + 1): 
        for w in range(W + 1): 
            if i == 0 or w == 0: 
                K[i][w] = 0
            elif wt[i - 1] <= w: 
                K[i][w] = max(val[i - 1] 
                + K[i - 1][w - wt[i - 1]], 
                            K[i - 1][w]) 
            else: 
                K[i][w] = K[i - 1][w] 

    # Resultado del Knapsack
    res = K[n][W] 
    #print(res) 
    
    #Nos movemos en la matriz para obtener
    #Los items que fueron seleccionados
    w = W 
    for i in range(n, 0, -1): 
        if res <= 0: 
            break
        if res == K[i - 1][w]: 
            continue
        else: 
            # Este item fue elegido 
            items.append((i - 1))
            
            # Como su peso fue incluido
            # su valor es reducido
            res = res - val[i - 1] 
            w = w - wt[i - 1]
            
    return items

## Algoritmo Greedy para Fractional Knapsack

In [11]:
# Un algoritmo greedy para el problema de knapsack fraccional
# Notemos que se ordena por v/w si que modifiquemos v o w, para que se pueda
# dar la salida con los indices que se quedaran en el knapsack al final

def Knapsack_frac(v, w, W):
    order = sort_proportion(v, w)   #Ordenamos por v/w (sort_proportion)          
    weight = 0.0                               
    value = 0.0                                
    knapsack = []                              
    n = len(v)
    i = 0                                   
    while (weight < W) and (i < n):
        # Si podemos ajustar todo el order[i]-ésimo item
        # agréguelo y actualice el peso y el valor
        if weight + w[order[i]] <= W:         
            knapsack.append((order[i], 1.0))   
            weight = weight + w[order[i]]
            value = value + v[order[i]]
        else:
        # De otra manera, calcula la fraccione que se ajuste
        # y agregue esta fraccion
            fraction = (W - weight) / w[order[i]]  
            knapsack.append((order[i], fraction))
            weight = W
            value = value + v[order[i]] * fraction
        i = i + 1
    #Regrese los items elegidos en el knapsack y sus respectivos valores
    return (knapsack, value)                       
  
# Ordenamos la lista en orden descendente por la razon de A[1] en N[i]
# pero en vez de re-ordenar A y B, guardamos el orden en 
# un arreglo a parte.
def sort_proportion(A, B):
    n = len(A)
    order = list(range(n))
    for i in range(n - 1, 0, -1):     
        for j in range(0, i):           
            if ((1.0 * A[order[j]]) / B[order[j]]) < ((1.0 * A[order[j+1]]) / B[order[j+1]]):
                #Se cambian "apuntadores" de los items
                temp = order[j]              
                order[j] = order[j+1]
                order[j+1] = temp
    return order

# Resultados

In [12]:
val = value_fr
wt = weight_fr 
W = int(W_f)
n = len(val) 
A = knapsack_dp(W, wt, val, n) 
res = 0
for i in A:
    res += weight_fr[i]
    print(name_fr[i])
x = float(len(A))
r = x//0.9
print('Con $',W_f,' se compraran ',x ,' KG de Fruta')
print('Alcanzaran cerca de: ',r, 'dias')
print('En total se gasto: ', res)

TEJOCOTE
SANDÍA
PLÁTANO TABASCO
PLÁTANO MORADO
PLÁTANO DOMINICO
PIÑA
PERÓN
PAPAYA
NARANJA AGRIA
NARANJA
MANZANA
MANDARINA
LIMÓN REAL
LIMÓN AGRIO
LIMA
GUAYABA
CAPULÍN
AGUACATE CRIOLLO
AGUACATE
Con $ 400.0  se compraran  19.0  KG de Fruta
Alcanzaran cerca de:  21.0 dias
En total se gasto:  396


In [13]:
val = value_ve
wt = weight_ve 
W = int(W_v)
n = len(val) 
B = knapsack_dp(W, wt, val, n) 
res_ve = 0
for i in B:
    res_ve += weight_ve[i]
    print(name_ve[i])
x = float(len(B))
r = x//0.9
print('Con $',W_v,' se compraran ',x ,' KG de Verdura')
print('Alcanzaran cerca de: ',r, 'dias')
print('En total se gasto: ', res_ve)

ZANAHORIA
VERDOLAGA
TOMATE VERDE
QUELITES
PEPINO
JITOMATE
HUAZONTLE
CEBOLLA CAMBRAY
CEBOLLA
CALABACITA
BERROS
ACELGA
Con $ 200.0  se compraran  12.0  KG de Verdura
Alcanzaran cerca de:  13.0 dias
En total se gasto:  200


In [14]:
w = weight_oi
v = value_oi
W = W_o
C = Knapsack_frac(v, w, W)[0]
total = 0
res_oi = 0
for tupla in C:
    i = tupla[0]
    frac = tupla[1]
    total += frac
    res_oi += weight_oi[i]*frac
    print(name_oi[i], '- ', frac , 'KG')
r_oi = 21*total
print('Con $',W_o,' se compraran ',total ,' KG de Semillas Oleaginosas')
print('Alcanzaran cerca de: ',r_oi, 'dias')
print('En total se gasto: ', res_oi)

GIRASOL -  1.0 KG
CHÍA -  1.0 KG
LINAZA -  1.0 KG
AJONJOLÍ -  1.0 KG
CACAHUATE -  1.0 KG
CALABAZA DE CASTILLA -  0.9076923076923077 KG
Con $ 400.0  se compraran  5.907692307692308  KG de Semillas Oleaginosas
Alcanzaran cerca de:  124.06153846153848 dias
En total se gasto:  400.0
